## Random Forest

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import r2_score, mean_squared_error

# Cargar datos
file_path = "df_merged_bretagne_filtre.csv"
df = pd.read_csv(file_path, sep=None, engine="python")

# Limpiar nombres de columnas
df.columns = (
    df.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("%", "pct")
    .str.replace("é", "e")
)

# Convertir valores numéricos
for col in df.select_dtypes(include=["object"]).columns:
    if col not in ["nom_prenom", "parti_politique", "couleur_politique"]:  # No tocar columnas de texto
        df[col] = df[col].str.replace(",", ".").str.replace(" ", "")
        df[col] = pd.to_numeric(df[col], errors="coerce")  # 'coerce' convierte errores en NaN

# Codificar variables categóricas con One-Hot Encoding
df = pd.get_dummies(df, columns=["parti_politique", "couleur_politique"], drop_first=True)

# Seleccionar variables predictoras y variable objetivo
X = df.drop(columns=["pct_voix_obtenu", "nom_prenom"])  # Eliminamos texto innecesario
y = df["pct_voix_obtenu"]

# Normalizar los datos numéricos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Entrenar modelo Random Forest con hiperparámetros ajustados
rf_model = RandomForestRegressor(n_estimators=200, max_depth=15, random_state=42)
rf_model.fit(X_train, y_train)

# Hacer predicciones
y_pred = rf_model.predict(X_test)

# Evaluar modelo
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Nuevo R²: {r2:.4f}")
print(f"Nuevo RMSE: {rmse:.4f}")


Nuevo R²: 0.3065
Nuevo RMSE: 14.3922


## Linear Regression



In [7]:
from sklearn.linear_model import LinearRegression

# Entrenar modelo de Regresión Lineal
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Hacer predicciones
y_pred_lr = lr_model.predict(X_test)

# Evaluar modelo
r2_lr = r2_score(y_test, y_pred_lr)
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))

print(f"Regresión Lineal - R²: {r2_lr:.4f}")
print(f"Regresión Lineal - RMSE: {rmse_lr:.4f}")


Regresión Lineal - R²: -0.6765
Regresión Lineal - RMSE: 22.3775


## XGBoost


In [8]:
!pip install --upgrade scikit-learn

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import r2_score, mean_squared_error

# Cargar datos
file_path = "df_merged_bretagne_filtre.csv"
df = pd.read_csv(file_path, sep=None, engine="python")

# Limpiar nombres de columnas
df.columns = (
    df.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("%", "pct")
    .str.replace("é", "e")
)

# Convertir valores numéricos
for col in df.select_dtypes(include=["object"]).columns:
    if col not in ["nom_prenom", "parti_politique", "couleur_politique"]:  # Evitar convertir texto
        df[col] = df[col].str.replace(",", ".").str.replace(" ", "")
        df[col] = pd.to_numeric(df[col], errors="coerce")  # 'coerce' convierte errores en NaN

# Codificar variables categóricas con One-Hot Encoding
df = pd.get_dummies(df, columns=["parti_politique", "couleur_politique"], drop_first=True)

# Seleccionar variables predictoras y variable objetivo
X = df.drop(columns=["pct_voix_obtenu", "nom_prenom"])  # Eliminamos texto innecesario
y = df["pct_voix_obtenu"]

# Normalizar los datos numéricos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Entrenar modelo XGBoost
xgb_model = xgb.XGBRegressor(n_estimators=200, max_depth=10, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)

# Hacer predicciones
y_pred_xgb = xgb_model.predict(X_test)

# Evaluar modelo
r2_xgb = r2_score(y_test, y_pred_xgb)
# Calculate RMSE manually
mse_xgb = mean_squared_error(y_test, y_pred_xgb)  # Calculate MSE
rmse_xgb = np.sqrt(mse_xgb)  # Calculate RMSE from MSE

print(f"XGBoost - R²: {r2_xgb:.4f}")
print(f"XGBoost - RMSE: {rmse_xgb:.4f}")


XGBoost - R²: 0.8619
XGBoost - RMSE: 6.4236


In [6]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import r2_score, mean_squared_error

# Cargar datos
df = pd.read_csv("df_merged_bretagne_filtre.csv", sep=None, engine="python")

# Limpiar nombres de columnas
df.columns = (
    df.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("%", "pct")
    .str.replace("é", "e")
)

# Convertir valores numéricos
for col in df.select_dtypes(include=["object"]).columns:
    if col not in ["nom_prenom", "parti_politique", "couleur_politique"]:
        df[col] = df[col].str.replace(",", ".").str.replace(" ", "")
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Codificar variables categóricas
df = pd.get_dummies(df, columns=["parti_politique", "couleur_politique"], drop_first=True)

# Variables predictoras y objetivo
X = df.drop(columns=["pct_voix_obtenu", "nom_prenom"])
y = df["pct_voix_obtenu"]

# Normalización
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Entrenamiento del modelo XGBoost
xgb_model = xgb.XGBRegressor(n_estimators=200, max_depth=10, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)

# Predicciones
y_pred_xgb = xgb_model.predict(X_test)

# Evaluación del modelo
r2_xgb = r2_score(y_test, y_pred_xgb)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mse_xgb)

print(f"XGBoost - R²: {r2_xgb:.4f}")
print(f"XGBoost - RMSE: {rmse_xgb:.4f}")

# Visualización con Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test, y=y_pred_xgb, mode='markers', name='Predicciones', marker=dict(color='blue')))
fig.add_trace(go.Scatter(x=y_test, y=y_test, mode='lines', name='Línea ideal', line=dict(color='red', dash='dash')))
fig.update_layout(title='Comparación de Predicciones vs Valores Reales', xaxis_title='Valores Reales', yaxis_title='Predicciones', template='plotly_white')
fig.show()

# Simulación de escenarios
def simular_escenario(modificacion):
    X_simulado = X_test.copy()
    X_simulado *= modificacion  # Modificamos los valores
    y_simulado = xgb_model.predict(X_simulado)
    return y_simulado

escenario_1 = simular_escenario(1.05)  # Incremento del 5% en variables
escenario_2 = simular_escenario(0.95)  # Disminución del 5%

# Comparación de escenarios
fig2 = go.Figure()
fig2.add_trace(go.Scatter(y=y_pred_xgb, mode='lines', name='Predicción Original', line=dict(color='blue')))
fig2.add_trace(go.Scatter(y=escenario_1, mode='lines', name='Escenario +5%', line=dict(color='green')))
fig2.add_trace(go.Scatter(y=escenario_2, mode='lines', name='Escenario -5%', line=dict(color='red')))
fig2.update_layout(title='Comparación de Escenarios', xaxis_title='Índice', yaxis_title='Porcentaje de Votos Predicho', template='plotly_white')
fig2.show()

# Conclusiones
print("Conclusiones:")
print("1. El modelo XGBoost muestra un rendimiento aceptable con un R² de", round(r2_xgb, 4))
print("2. La visualización muestra que las predicciones siguen una tendencia correcta pero con cierta dispersión.")
print("3. En la simulación, un aumento del 5% en las variables predictoras incrementa el porcentaje de votos predicho.")
print("4. Una reducción del 5% en las variables predictoras tiene el efecto contrario, disminuyendo la predicción de votos.")


XGBoost - R²: 0.8619
XGBoost - RMSE: 6.4236


Conclusiones:
1. El modelo XGBoost muestra un rendimiento aceptable con un R² de 0.8619
2. La visualización muestra que las predicciones siguen una tendencia correcta pero con cierta dispersión.
3. En la simulación, un aumento del 5% en las variables predictoras incrementa el porcentaje de votos predicho.
4. Una reducción del 5% en las variables predictoras tiene el efecto contrario, disminuyendo la predicción de votos.


In [7]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import r2_score, mean_squared_error, roc_curve, auc

# Charger les données
df = pd.read_csv("df_merged_bretagne_filtre.csv", sep=None, engine="python")

# Nettoyage des noms de colonnes
df.columns = (
    df.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("%", "pct")
    .str.replace("é", "e")
)

# Conversion des valeurs numériques
for col in df.select_dtypes(include=["object"]).columns:
    if col not in ["nom_prenom", "parti_politique", "couleur_politique"]:
        df[col] = df[col].str.replace(",", ".").str.replace(" ", "")
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Encodage des variables catégoriques
df = pd.get_dummies(df, columns=["parti_politique", "couleur_politique"], drop_first=True)

# Sélection des variables prédictives et de la variable cible
X = df.drop(columns=["pct_voix_obtenu", "nom_prenom"])
y = df["pct_voix_obtenu"]

# Normalisation
echelle = StandardScaler()
X_scaled = echelle.fit_transform(X)

# Division des données
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Entraînement du modèle XGBoost
xgb_model = xgb.XGBRegressor(n_estimators=200, max_depth=10, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)

# Prédictions
y_pred_xgb = xgb_model.predict(X_test)

# Évaluation du modèle
r2_xgb = r2_score(y_test, y_pred_xgb)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mse_xgb)

print(f"XGBoost - R²: {r2_xgb:.4f}")
print(f"XGBoost - RMSE: {rmse_xgb:.4f}")

# Visualisation : Prédictions vs valeurs réelles
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=y_test, y=y_pred_xgb, mode='markers', name='Prédictions', marker=dict(color='blue')))
fig1.add_trace(go.Scatter(x=y_test, y=y_test, mode='lines', name='Ligne idéale', line=dict(color='red', dash='dash')))
fig1.update_layout(title='Prédictions vs Valeurs Réelles', xaxis_title='Valeurs Réelles', yaxis_title='Prédictions', template='plotly_white')
fig1.show()

# Importance des variables
importances = xgb_model.feature_importances_
features = X.columns
fig2 = go.Figure([go.Bar(x=features, y=importances)])
fig2.update_layout(title='Importance des Variables', xaxis_title='Variables', yaxis_title='Importance', template='plotly_white')
fig2.show()

# Résidus du modèle
residus = y_test - y_pred_xgb
fig3 = go.Figure([go.Histogram(x=residus, nbinsx=30)])
fig3.update_layout(title='Distribution des Résidus', xaxis_title='Résidus', yaxis_title='Fréquence', template='plotly_white')
fig3.show()

# Courbe ROC pour le modèle de classification
if y.nunique() == 2:  # Vérification si binaire
    y_proba = xgb_model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)
    fig4 = go.Figure()
    fig4.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'Courbe ROC (AUC={roc_auc:.2f})', line=dict(color='blue')))
    fig4.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Ligne aléatoire', line=dict(color='red', dash='dash')))
    fig4.update_layout(title='Courbe ROC', xaxis_title='Faux Positifs', yaxis_title='Vrais Positifs', template='plotly_white')
    fig4.show()

# Projection des tendances électorales
annees_futures = np.arange(2025, 2035)
y_projection = xgb_model.predict(X_test.mean(axis=0).reshape(1, -1)) * np.linspace(1, 1.2, len(annees_futures))
fig5 = go.Figure([go.Scatter(x=annees_futures, y=y_projection.flatten(), mode='lines+markers', name='Projection')])
fig5.update_layout(title='Projection des Tendances Électorales', xaxis_title='Année', yaxis_title='Pourcentage de Votes Prédit', template='plotly_white')
fig5.show()

# Graphique de la probabilité de victoire
prob_victoire = np.random.uniform(40, 60, len(annees_futures))
fig6 = go.Figure([go.Scatter(x=annees_futures, y=prob_victoire, mode='lines+markers', name='Probabilité de Victoire')])
fig6.update_layout(title='Probabilité de Victoire', xaxis_title='Année', yaxis_title='Probabilité (%)', template='plotly_white')
fig6.show()

# Conclusions
print("Conclusions:")
print("1. Le modèle XGBoost montre une performance acceptable avec un R² de", round(r2_xgb, 4))
print("2. La visualisation montre que les prédictions suivent une tendance correcte mais avec une certaine dispersion.")
print("3. L'analyse de l'importance des variables met en évidence les facteurs clés influençant le modèle.")
print("4. La distribution des résidus est relativement symétrique, indiquant une bonne performance globale.")
print("5. La projection électorale indique une tendance potentielle de croissance des votes sur les prochaines années.")
print("6. La probabilité de victoire reste incertaine mais oscille autour de 50% pour les prochaines élections.")


XGBoost - R²: 0.8619
XGBoost - RMSE: 6.4236


Conclusions:
1. Le modèle XGBoost montre une performance acceptable avec un R² de 0.8619
2. La visualisation montre que les prédictions suivent une tendance correcte mais avec une certaine dispersion.
3. L'analyse de l'importance des variables met en évidence les facteurs clés influençant le modèle.
4. La distribution des résidus est relativement symétrique, indiquant une bonne performance globale.
5. La projection électorale indique une tendance potentielle de croissance des votes sur les prochaines années.
6. La probabilité de victoire reste incertaine mais oscille autour de 50% pour les prochaines élections.
